In [39]:
import numpy as np
import matplotlib.pyplot as plt
import struct
import math
from statistics import mean, stdev

import qiskit
from qiskit import *

import pennylane as qml
from pennylane import numpy as pnp

import warnings
warnings.filterwarnings('ignore')

In [47]:
iterations = 100
img_size = 10
binary_MSE = []
amplitude_MSE = []

for i in range(0, iterations):
    
    initial, visibilities = generate_img(img_size) 
    
    # BINARY
    binary_readout = binary(visibilities)
    
    # AMPLITUDE
    n_n = img_size*img_size
    inp = visibilities.flatten()
    norm = qml.math.sum(qml.math.abs(inp) ** 2)
    number_of_qbits = math.ceil(math.log2(n_n))
    wires = range(number_of_qbits)
    amp_dev = qml.device('default.qubit', wires)
    @qml.qnode(amp_dev)
    def amp_encoding(data):
        qml.AmplitudeEmbedding(data, wires, normalize=True, pad_with=0.)
        return qml.state()
    readout = amp_encoding(inp).astype('complex64')
    readout.numpy()
    readout = readout*qml.math.sqrt(norm) 
    readout = readout[:n_n] 
    readout = np.array(readout).reshape(img_size , img_size)
    amplitude_readout = np.fft.ifft2(readout).astype('complex64')
    
    binary_mse = MSE(initial, binary_readout)
    amplitude_mse = MSE(initial, amplitude_readout)
    
    binary_MSE.append(binary_mse)
    amplitude_MSE.append(amplitude_mse)

print('Binary MSEs: '+ str(binary_MSE))
print()
print('Amplitude MSEs: '+ str(amplitude_MSE))
print()

binary_mean = mean(binary_MSE)
print('Binary mean: '+ '\033[1m' + str(binary_mean) + '\033[0m')
#binary_std = stdev(binary_MSE)
#print(binary_std)
amplitude_mean = mean(amplitude_MSE)
print('Amplitude mean: '+ '\033[1m' + str(amplitude_mean) + '\033[0m')
#amplitude_std = stdev(amplitude_MSE)
#print(amplitude_std)

Binary MSEs: [2.6019692e-13, 1.9765338e-13, 6.987292e-14, 2.0895265e-13, 2.872721e-14, 9.73631e-15, 1.326739e-13, 4.160996e-14, 7.452692e-14, 1.4699415e-13, 1.2955273e-13, 5.4262015e-14, 6.5395755e-14, 1.1971939e-13, 1.375708e-13, 2.0989704e-13, 6.2341605e-14, 1.153812e-13, 1.0070162e-13, 6.255921e-14, 2.0045075e-13, 1.9935803e-13, 3.2564928e-13, 3.3275122e-15, 1.2257525e-13, 1.2799806e-13, 9.648304e-14, 2.4253505e-13, 1.2603891e-13, 3.0033484e-14, 3.092818e-13, 2.8762816e-13, 1.1595472e-13, 1.1961553e-13, 5.6663224e-14, 5.734023e-14, 1.5388897e-13, 5.1856617e-14, 1.07900695e-13, 8.2215864e-14, 5.6592982e-14, 3.6379784e-14, 3.7552768e-13, 6.059959e-14, 2.524118e-13, 1.9992425e-13, 6.733919e-14, 1.9423298e-13, 4.3667674e-15, 3.7313737e-15, 1.3350108e-13, 1.0188051e-13, 3.6278638e-14, 2.276126e-13, 1.7135471e-13, 2.6891373e-14, 1.223191e-13, 1.6828962e-13, 2.0597849e-13, 2.3872283e-13, 9.8630685e-14, 3.3696144e-13, 1.5756588e-13, 1.5094585e-13, 2.449763e-13, 2.2576879e-13, 1.13544875e-13

In [40]:
# float to binary 
def float_to_bin_real(num):
    return format(struct.unpack('!I', struct.pack('!f', num))[0], '032b')
def float_to_bin_imag(num):
    return format(struct.unpack('!I', struct.pack('!f', num))[0], '032b')

# binary to float
def bin_to_float_real(binary):
    return struct.unpack('!f',struct.pack('!I', int(binary, 2)))[0]
def bin_to_float_imag(binary):
    return struct.unpack('!f',struct.pack('!I', int(binary, 2)))[0]

# reverse a string (for the measurement step readout)
def reverse(string):
    string = string[::-1]
    return string

In [41]:
def generate_img(n):

    sky_image = np.zeros((n, n), dtype='complex64')
    mask = np.zeros((n, n), dtype='bool')
    stars = np.random.randint(1, 7) 
    radius = 1 

    for i in range(0, stars):
        index_i = np.random.randint(0, n-radius)
        index_j = np.random.randint(0, n-radius)
        for j in range(0, radius):
            for k in range(0, radius):
                sky_image[index_i+j][index_j+k] = complex(np.random.rand()*100, np.random.rand()*100)
                mask[index_i+j][index_j+k] = True
            
    sky_image.real += np.random.randn(n,n)/100
    sky_image.imag += np.random.randn(n,n)/100
    
    visibilities = np.fft.fft2(sky_image).astype('complex64')
    
    return sky_image, visibilities

In [42]:
def binary_encode(qc, binary, off_set):
    
    for i in range(0, len(binary)):
        qc.reset(off_set+i) 

        if binary[i]=='1':
            qc.x(off_set+i) 
    
    off_set += len(binary)

    return off_set

In [43]:
def binary(img):
    
    number_of_pixels = img.shape[0]*img.shape[1]
    number_of_bits_real = 32
    number_of_bits_imag = 32
    number_of_bits = number_of_bits_real + number_of_bits_imag
    number_of_qbits = number_of_pixels*number_of_bits
    off_set = 0

    qc = QuantumCircuit(number_of_qbits)
    for i in range(0, img.shape[0]):
        for j in range(0, img.shape[1]):
                binary_real = float_to_bin_real(img[i, j].real)
                binary_imag = float_to_bin_imag(img[i, j].imag)
                binary = binary_real+binary_imag
                off_set = binary_encode(qc, binary, off_set)
                
    qc.measure_all()
    backend = Aer.get_backend('aer_simulator') 
    job = backend.run(qc, shots=1, memory=True) 
    output = job.result().get_memory()[0]
    out = reverse(output)
    
    chunks_real = []
    chunks_imag = []
    for i in range(0, number_of_pixels):
        chunks_real.append(out[number_of_bits*i:(number_of_bits*i)+number_of_bits_real]) 
        chunks_imag.append(out[(number_of_bits*i)+number_of_bits_imag:(number_of_bits*i)+number_of_bits_imag+number_of_bits_imag]) 

    readout = []
    for i in range(0, len(chunks_real)):
        readout.append(complex(bin_to_float_real(chunks_real[i]), bin_to_float_imag(chunks_imag[i]))) 

    readout = np.array(readout).reshape(img.shape[0] , img.shape[1]).astype('complex64')
    readout = np.fft.ifft2(readout).astype('complex64') #IFT
    
    return readout

In [44]:
def MSE(img1, img2):
    real_mse = ((img1.real - img2.real)**2).mean()
    imag_mse = ((img1.imag - img2.imag)**2).mean()
    mse = real_mse + imag_mse
    
    return mse